In [31]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame

import matplotlib.pyplot as plt                     # Basic visualisation
import seaborn as sns                               # Advanced data visualisation

from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

We hebben twee datasets. Demographic en Student. Dit zijn twee datasets die dezelfde studenten bevatten, maar enkele andere waarden. Met een inner join kunnen we deze twee dataframes gaan samenvoegen of mergen.
1. We voegen de twee dataframes samen.
2. We verwijderen alle rijen die nulwaarden bevatten.
3. We kunnen niet werken met 'standard' of 'free/reduced'. Deze waarden moeten we omzetten naar numerieke waarden zoals 0 en 1.

In [32]:
demographic = pd.read_csv('https://raw.githubusercontent.com/jdecorte/machinelearning/main/datasets/demographic.csv')
student = pd.read_csv('https://raw.githubusercontent.com/jdecorte/machinelearning/main/datasets/studentscore.csv')

df = pd.merge(student, demographic, how='inner', on='StudentID')

In [33]:
df.head()

,StudentID,lunch,test preparation course,math score,gender,race/ethnicity,parental level of education
0,0,standard,none,72,female,group B,bachelor's degree
1,1,standard,completed,69,female,group C,some college
2,2,standard,none,90,female,group B,master's degree
3,3,free/reduced,none,47,male,group A,associate's degree
4,4,standard,none,76,male,group C,some college


In [34]:
df = df.dropna()

In [35]:
dictmap = {'standard':0, 'free/reduced':1}
df['lunch'] = df['lunch'].map(dictmap)

dictmap = {'none':0, 'completed':1}
df['test preparation course'] = df['test preparation course'].map(dictmap)

dictmap = {'female':0, 'male':1}
df['gender'] = df['gender'].map(dictmap)

df.head()


,StudentID,lunch,test preparation course,math score,gender,race/ethnicity,parental level of education
0,0,0,0,72,0,group B,bachelor's degree
1,1,0,1,69,0,group C,some college
2,2,0,0,90,0,group B,master's degree
3,3,1,0,47,1,group A,associate's degree
4,4,0,0,76,1,group C,some college


## Het model bouwen

1. We gaan een model bouwen op basis van de math score. Dit wordt dus onze y. 'ID' hebben we niet meer nodig. Die kunnen we ook droppen.
2. We splitsen de dataset in een training- en een testset.

In [36]:
X = student.drop(['StudentID','math score'], axis=1)
y = student['math score']

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)

In [38]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

750
250
750
250


In [39]:
numerical_ix = X.select_dtypes(include=['int32','int64','float64']).columns
tekst_ix = X.select_dtypes(include=['object', 'string']).columns

## Pipeline maken

1. We gaan eerst kijken om de categoriëen aan te passen met een OneHotEncoder. Hierbij moeten we wel opletten dat we remainder='passthrough' bij de columntransformer noteren. Dit zorgt er voor dat de kolommen, die niet in de lijst van transformers staan, gewoon worden doorgegeven in plaats van weggeworpen te worden. Als we deze kolommen toch niet nodig hebben mogen we dit weg laten.

2. We gaan een dictionary van regressors maken. We houden twee modellen bij: een SVM Regressiemodel en een SVM Regressiemodel met een tweedegraads polynomiale kernel. De bedoeling is dat we zometeen over deze dictionary gaan itereren.

3. We itereren over deze dictionary. Een item in de dictionary vragen we op met .items(). Voor iedere regressor gaan we een Pipeline opbouwen met daarin de columntransformer en de regressor die op dat moment aan bod komt. 

3. We trainen dat model met de fit-functie.

4. We berekenen de MAE voor ieder model. Deze printen we uit.

In [40]:
from sklearn.pipeline import Pipeline

# Passthrough --> de resterende kolommen ongewijzigd laten.
col_transform = ColumnTransformer(transformers=[('cat', OneHotEncoder(), tekst_ix)], remainder='passthrough')

In [41]:
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

regressors = {
    # voor lineaire SVM regressie
    "svm_reg": LinearSVR(epsilon=1.5, random_state=42),
    
    # voor de polynomiale kernel
    "svm_poly_reg": SVR(kernel='poly', degree=2, C=100, epsilon=0.1, gamma='scale')
}

In [42]:
for key, regressor in regressors.items():
    pipeline = Pipeline([
        ('prep',col_transform),
        (key, regressor)
    ])

    pipeline.fit(X_train, y_train)

    mae = np.mean(cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_absolute_error', cv=3))
    print(f'{key} --> mae = {-mae}')

svm_reg --> mae = 11.160368041638351
svm_poly_reg --> mae = 11.135460334449169


## Gridsearch voor een lineair model.

In [43]:
from sklearn.model_selection import GridSearchCV

svr_params = {
    'epsilon': [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5]
    }

grid_svc = GridSearchCV(LinearSVR(random_state=42), svr_params, scoring='neg_mean_absolute_error', cv=3)

pipeline = Pipeline([
    ('prep',col_transform),
    ('grid_svc', grid_svc)
    ])
pipeline.fit(X_train, y_train)

# best estimator
print(f"best estimator = {grid_svc.best_estimator_}")
# best score
print(f"best score = {grid_svc.best_score_}")

best estimator = LinearSVR(epsilon=2.5, random_state=42)
best score = -11.106819596291984


C:\Users\dylan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\dylan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\dylan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\dylan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed t

## Grid-search met niet-lineaire regressie

In [29]:
svr_params = {'epsilon': [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5], "kernel": ["rbf", "poly", "sigmoid", "linear"], "degree": [1, 2, 3]}

grid_svc = GridSearchCV(SVR(), svr_params, scoring='neg_mean_absolute_error', cv=3)
pipeline = Pipeline([('prep',col_transform), ('grid_svc', grid_svc)])
pipeline.fit(X_train, y_train)

# best estimator
print(f"best estimator = {grid_svc.best_estimator_}")
# best score
print(f"best score = {grid_svc.best_score_}")

best estimator = SVR(degree=1, epsilon=3.0, kernel='poly')
best score = -11.065337239583334
